In [16]:
from openai import OpenAI

client = OpenAI()

tools = [
    {
        "type": "function",
        "function": {
            "name": "extract_address",
            "description": "Extracts an address from a user query.",
            "parameters": {
                "type": "object",
                "properties": {
                    "street": {"type": "string"},
                    "city": {"type": "string"},
                    "state": {"type": "string"},
                    "postal_code": {"type": "string"},
                },
                "required": ["street", "city", "state", "postal_code"]
            }
        }
    }
]

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {"role": "user", "content": "Gdzie mieszka Smok wawelski?"}
    ],
    tools=tools,
    tool_choice={"type": "function", "function": {"name": "extract_address"}}
)

tool_call = response.choices[0].message.tool_calls[0]
arguments = tool_call.function.arguments

import json
address = json.loads(arguments)

print(address["city"], address["street"], address["state"], address["postal_code"])

Kraków Wawel 5 małopolskie 31-001


In [ ]:
pip install pydantic

In [ ]:
pip install instructor

In [17]:
from pydantic import BaseModel

class Address(BaseModel):
    street: str
    city: str
    state: str
    postal_code: str

In [18]:
from openai import OpenAI
import instructor

client = instructor.from_openai(OpenAI())

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": "Gdzie mieszka Smok wawelski?"
        }
    ],
    response_model=Address,
)

print(response.city, response.street, response.state, response.postal_code)

Kraków Bulwar Czerwieński Małopolska 31-069


In [19]:
from pydantic import field_validator

class AddressValidated(BaseModel):
    street: str
    city: str
    state: str
    postal_code: str

    @field_validator('city')
    def must_be_capital(cls, v: str) -> str:
        if v != v.upper():
            raise ValueError(f"All letters should be capitalized: `{v}`")
        return v

In [20]:
from openai import OpenAI
import instructor

client = instructor.from_openai(OpenAI())

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": "Gdzie mieszka Smok wawelski?"
        }
    ],
    max_retries=1,
    response_model=AddressValidated,
)

print(response.city, response.street, response.state, response.postal_code)

InstructorRetryException: 1 validation error for AddressValidated
city
  Value error, All letters should be capitalized: `Kraków` [type=value_error, input_value='Kraków', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

In [ ]:
from openai import OpenAI
import instructor

client = instructor.from_openai(OpenAI())

response = client.chat.completions.create(
    model="gpt-4o",
    messages=[
        {
            "role": "user",
            "content": "Gdzie mieszka Smok wawelski?"
        }
    ],
    max_retries=2,
    response_model=AddressValidated,
)

print(response.city, response.street, response.state, response.postal_code)